In [ ]:
texts = [
   "la atencion en este sitio es mala ya quiero volver a casa",
   "eficiente y comodo",
   "Cálida bienvenida, personal eficiente",
   "este sitio es muy bueno y la atencion es de lo mejor",
   "la atencion en este sitio es mala ya quiero volver a casa",
   "espero nunca jamas volver a un lugar tan despreciable y triste como este",
   "Me encanta este producto, es realmente fantástico.",
   "Estoy muy decepcionado con el servicio.",
   "La película fue bastante aburrida.",
   "¡Qué experiencia tan increíble!",
   "No volveré a comprar aquí, pésima atención al cliente."
]
